In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [26]:
import codecs

with codecs.open('news/NewsArticles.csv', 'r', 'utf-8', 'ignore') as f:
    df = pd.read_csv(f)

df = df.iloc[:,0:6]

In [37]:
df['title']

51                              #MyFreedomDay: Students stand up to slavery on March 14
103                                                   Your pictures: Broken resolutions
293    Germany's new astronaut Matthias Maurer eyes 2020 mission and Chinese space ties
294                                  Poland debates approach to new World War II museum
295                          Why the world is turning to Hannah Arendt to explain Trump
296                           Lady Gaga, Merkel and heretics meet in monumental artwork
297                            Salt poses threat to Senegal's Sin Saloum Delta wetlands
298       The wetlands along the river Rhine struggle to recover from human engineering
299                Beyonce announces twins, giving fans appreciated break from politics
929                                         Only 30 left: Vaquita numbers keep dropping
Name: title, dtype: object

In [34]:
# 2017/2/2（10本）の記事からなるテストデータを作成
df = df[df['publish_date'] == '2017/2/2']
len(df)

10

In [52]:
#　センテンスのリストを作る
sentence_list = []
for content in df['text']:
    if type(content) is str:
        sentences = tokenize.sent_tokenize(content)
        for sentence in sentences:
            sentence_list.append(sentence)
    else:
        pass

In [55]:
# OpenIEにかける
from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for sentence in sentence_list:
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

In [56]:
rows = []
for extractions in extractions_list:
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [sentence, arg1, rel, arg2s, confidence]
        rows.append(row)

In [57]:
kb_df = pd.DataFrame(rows, columns = ['sentence', 'arg1', 'rel', 'arg2s', 'confidence'])
kb_df.head()

,sentence,arg1,rel,arg2s,confidence
0,"Although slavery is outlawed in every country, criminals earn more than $150 Billion every year from enslaving people.",slavery,is outlawed,in every country,0.894243
1,But you can do something about it.,you,can do,something about it,0.464930
2,"On MARCH 14, 2017, we want YOU and your school to join us for #MyFreedomDay.",we,want,"YOU to join us for # MyFreedomDay On MARCH 14 , 2017",0.495379
3,We're asking students to organize events at their school on the day to highlight modern slavery and celebrate freedom.,We,'re asking,students to organize events at their school on the day to highlight modern slavery,0.417635
4,"The schools that show the most spirit in fighting slavery will be featured on CNN TV, CNN.com and CNN Facebook.",The schools,show,the most spirit in fighting slavery,0.912624


In [58]:
len(kb_df)

261

In [59]:
from db import connect
engine = connect()
kb_df.to_sql(name='news',con=engine,if_exists='replace',index=None)